### Starting a LJ simulation
- initialize polymer system using a DPD potential
- starting a simulation with a LJ potential
- angles and dihedrals added to the DPD workflow
- Enhancement: make angles and dihedrals optional

In [1]:
import matplotlib
import numpy as np  
import gsd, gsd.hoomd 
import hoomd 
import freud
import time
import matplotlib_inline
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use("ggplot")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [4]:
def initialize_snapshot_rand_walk(num_pol, num_mon, density=0.85, bond_length=1.0, buffer=0.1):
    '''
    Create a HOOMD snapshot of a cubic box with the number density given by input parameters.
    '''    
    N = num_pol * num_mon
    L = np.cbrt(N / density)  # Calculate box size based on density
    positions = np.zeros((N, 3))
    for i in range(num_pol):
        start = i * num_mon
        positions[start] = np.random.uniform(low=(-L/2),high=(L/2),size=3)
        for j in range(num_mon - 1):
            delta = np.random.uniform(low=(-bond_length/2),high=(bond_length/2),size=3)
            delta /= np.linalg.norm(delta)*bond_length
            positions[start+j+1] = positions[start+j] + delta
    positions = pbc(positions,[L,L,L])
    bonds = []
    for i in range(num_pol):
        start = i * num_mon
        for j in range(num_mon - 1):
            bonds.append([start + j, start + j + 1])
    angles = []
    for i in range(num_pol):
        start = i * num_mon
        for j in range(num_mon - 2):
            angles.append([start + j, start + j + 1,start + j + 2])
    dihedrals = []
    for i in range(num_pol):
        start = i * num_mon
        for j in range(num_mon - 3):
            dihedrals.append([start + j, start + j + 1,start + j + 2,start + j + 3])
    bonds = np.array(bonds)
    frame = gsd.hoomd.Frame()
    frame.particles.types = ['A']
    frame.particles.N = N
    frame.particles.position = positions
    frame.bonds.N = len(bonds)
    frame.bonds.group = bonds
    frame.bonds.types = ['b']
    frame.angles.N = (num_mon - 2)*num_pol
    frame.angles.types = ["A-A-A"]
    frame.angles.group = angles
    frame.dihedrals.N = (num_mon - 3)*num_pol
    frame.dihedrals.types = ["A-A-A-A"]
    frame.dihedrals.group = dihedrals
    frame.configuration.box = [L, L, L, 0, 0, 0]
    return frame

def pbc(d,box):
    for i in range(3):
        a = d[:,i]
        pos_max = np.max(a)
        pos_min = np.min(a)
        while pos_max > box[i]/2 or pos_min < -box[i]/2:
            a[a < -box[i]/2] += box[i]
            a[a >  box[i]/2] -= box[i]
            pos_max = np.max(a)
            pos_min = np.min(a)
    return d

def check_bond_length_equilibration(snap,num_mon,num_pol,max_bond_length=1.1,min_bond_length=0.9): #todo
    frame_ds = []
    for j in range(num_pol):
        idx = j*num_mon
        d1 = snap.particles.position[idx:idx+num_mon-1] - snap.particles.position[idx+1:idx+num_mon]
        bond_l = np.linalg.norm(pbc(d1,snap.configuration.box),axis=1)
        frame_ds.append(bond_l)
    max_frame_bond_l = np.max(np.array(frame_ds))
    min_frame_bond_l = np.min(np.array(frame_ds))
    print("max: ",max_frame_bond_l," min: ",min_frame_bond_l)
    if max_frame_bond_l <= max_bond_length and min_frame_bond_l >= min_bond_length:
        print("Bond's Relaxed.")
        return True
    if max_frame_bond_l > max_bond_length or min_frame_bond_l < min_bond_length:
        return False

def check_inter_particle_distance(snap,minimum_distance=0.95):
    '''
    Check particle separations.
    
    '''
    positions = snap.particles.position
    box = snap.configuration.box
    aq = freud.locality.AABBQuery(box,positions)
    aq_query = aq.query(
        query_points=positions,
        query_args=dict(r_min=0.0, r_max=minimum_distance, exclude_ii=True),
    )
    nlist = aq_query.toNeighborList()
    if len(nlist)==0:
        print("Inter-particle separation reached.")
        return True
    else:
        return False

def run_one(A=1000,gamma=1000,k=1000,num_pol=100,num_mon=10,kT=1.0,r_cut = 1.15,bond_l=1.0,dt=0.001,density=0.8,particle_spacing = 1.05
):
    print(num_pol*num_mon)
    print(f"\nRunning with A={A}, gamma={gamma}, k={k}, "
          f"num_pol={num_pol}, num_mon={num_mon}")
    start_time = time.perf_counter()
    frame = initialize_snapshot_rand_walk(num_pol, num_mon, density=density)
    build_stop = time.perf_counter()
    print("Total build time: ", build_stop-start_time)
    harmonic = hoomd.md.bond.Harmonic()
    harmonic.params["b"] = dict(r0=bond_l, k=k)
    integrator = hoomd.md.Integrator(dt=dt)
    integrator.forces.append(harmonic)
    simulation = hoomd.Simulation(device=hoomd.device.auto_select(), seed=np.random.randint(65535))# TODO seed
    simulation.operations.integrator = integrator 
    simulation.create_state_from_snapshot(frame)
    const_vol = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All())
    integrator.methods.append(const_vol)
    nlist = hoomd.md.nlist.Cell(buffer=0.4)
    simulation.operations.nlist = nlist
    DPD = hoomd.md.pair.DPD(nlist, default_r_cut=r_cut, kT=kT)
    DPD.params[('A', 'A')] = dict(A=A, gamma=gamma)
    integrator.forces.append(DPD)
    
    simulation.run(0)
    simulation.run(1000)
    snap=simulation.state.get_snapshot()
    N = num_pol*num_mon
    time_factor = N/90000
    
    while not check_bond_length_equilibration(snap,num_mon, num_pol,max_bond_length=particle_spacing): #TODO - work with snapshot instead of gsd?
        check_time = time.perf_counter()
        if (check_time-start_time) > 60*time_factor:
            return num_pol*num_mon, 0
        simulation.run(1000)
        snap=simulation.state.get_snapshot()

    while not check_inter_particle_distance(snap,minimum_distance=0.95):
        check_time = time.perf_counter()
        if (check_time-start_time) > 60*time_factor:
            return num_pol*num_mon, 0
        simulation.run(1000)
        snap=simulation.state.get_snapshot()
        
    end_time = time.perf_counter()
    return snap, end_time - start_time
    
dpd_final_frame,s = run_one(A=5000,gamma=800,k=25000,num_pol=50,num_mon=50) #A=1000 seems fast, cranking up gamma to 800 speeds up, cranking up k more does too,
#no difference between k=10k and 20k
#OK, with these parameters, we're finishing in under 100 steps
print(f"Finished in time = {s:.2f}s")

2500

Running with A=5000, gamma=800, k=25000, num_pol=50, num_mon=50
Total build time:  0.020886166021227837
max:  1.0679491982539648  min:  0.9343710017530248
max:  1.051302302583204  min:  0.9535298547148537
max:  1.0367820751208687  min:  0.9553678673446349
Bond's Relaxed.
Inter-particle separation reached.
Finished in time = 0.78s


In [5]:
random_seed=24
forces = []
# Create pair force:
nlist = hoomd.md.nlist.Cell(buffer=0.40, exclusions=["bond"])
lj = hoomd.md.pair.LJ(nlist=nlist)
lj.params[('A', 'A')] = dict(epsilon=1.0, sigma=1.0)
lj.r_cut[('A', 'A')] = 1.2
forces.append(lj)
# Create FENE bond force:
fene_bond = hoomd.md.bond.FENEWCA()
fene_bond.params['b'] = dict(
    k=30,
    r0=1.05,
    epsilon=1.0,
    sigma=1.0,
    delta=0,
)
forces.append(fene_bond)

integrator_lj = hoomd.md.Integrator(dt=0.001)
integrator_lj.forces = forces
LJ_sim = hoomd.Simulation(device=hoomd.device.auto_select(), seed=random_seed)
LJ_sim.create_state_from_snapshot(snapshot=dpd_final_frame)
LJ_sim.operations.integrator = integrator_lj
const_vol = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All())
integrator_lj.methods.append(const_vol)

gsd_out = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(10), 
    mode='wb',
    dynamic=['property','momentum'],
    filename='lj_sim.gsd',
    truncate=False)
LJ_sim.operations.writers.append(gsd_out)

LJ_sim.run(0)
LJ_sim.run(100)
gsd_out.flush()
#snap=simulation.state.get_snapshot()

In [6]:
#adding in angles
harmonic_angle = hoomd.md.angle.Harmonic()
harmonic_angle.params["A-A-A"] = dict(k=3.0, t0=1.0)
integrator_lj.forces.append(harmonic_angle)
LJ_sim.run(100)
gsd_out.flush()

In [8]:
#adding dihedrals
dihedral = hoomd.md.dihedral.Periodic()
dihedral.params["A-A-A-A"] = dict(k=3.0, d=-1, n=3, phi0=0)
integrator_lj.forces.append(dihedral)
LJ_sim.run(100)
gsd_out.flush()